In [ ]:
# !pip install tensorflow

In [ ]:
# !pip install matplotlib

In [ ]:
import pandas as pd
import numpy as np
import h5py
import glob
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf

In [ ]:
path_single = r"/kaggle/input/landslide-data/img/image_2000.h5"
path_single_mask = r'/kaggle/input/landslide-data/mask/mask_2000.h5'

In [ ]:
f_data = np.zeros((1, 128,128, 3))
with h5py.File(path_single) as hdf:
    ls = list(hdf.keys())
    print("ls", ls)
    data = np.array(hdf.get('img'))
    print("input data shape:", data.shape)
    plt.imshow(data[:, :, 3:0:-1])
    plt.show()

    data_red = data[:, :, 3]
    data_green = data[:, :, 2]
    data_blue = data[:, :, 1]
    data_nir = data[:, :, 7]
    data_rgb = data[:, :, 3:0:-1]
    data_ndvi = np.divide(data_nir - data_red,np.add(data_nir, data_red))
    f_data[0, :, :, 0] =data_ndvi
    f_data[0, :, :, 1] = data[:, :, 12]
    f_data[0, :, :, 2] = data[:, :, 13]

    print("data ndvi shape ", data_ndvi.shape, "f_data shape: ", f_data.shape)
    plt.imshow(data_ndvi)

In [ ]:
with h5py.File(path_single_mask) as hdf:
    ls = list(hdf.keys())
    print("ls", ls)
    data = np.array(hdf.get('mask'))
    print("input data shape:", data.shape)
    plt.imshow(data)

In [ ]:
TRAIN_PATH = r"/kaggle/input/landslide-data/img/*.h5"
TRAIN_MASK = r'/kaggle/input/landslide-data/mask/*.h5'

TRAIN_XX = np.zeros((3799, 128, 128, 6))
TRAIN_YY = np.zeros((3799, 128, 128, 1))
all_train = sorted(glob.glob(TRAIN_PATH))
all_mask = sorted(glob.glob(TRAIN_MASK))

In [ ]:
#testing for google colab GPU
import tensorflow as tf
tf.test.gpu_device_name()

In [ ]:
for i, (img, mask) in enumerate(zip(all_train, all_mask)):
    #print(i, img, mask)
    with h5py.File(img) as hdf:
        ls = list(hdf.keys())
        data = np.array(hdf.get('img'))

        # assign 0 for the nan value
        data[np.isnan(data)] = 0.000001

        # to normalize the data
        mid_rgb = data[:, :, 1:4].max() / 2.0
        mid_slope = data[:, :, 12].max() / 2.0
        mid_elevation = data[:, :, 13].max() / 2.0

        # ndvi calculation
        data_red = data[:, :, 3]
        data_nir = data[:, :, 7]
        data_ndvi = np.divide(data_nir - data_red,np.add(data_nir, data_red))

        # final array
        TRAIN_XX[i, :, :, 0] = 1 - data[:, :, 3] / mid_rgb  #RED
        TRAIN_XX[i, :, :, 1] = 1 - data[:, :, 2] / mid_rgb #GREEN
        TRAIN_XX[i, :, :, 2] = 1 - data[:, :, 1] / mid_rgb #BLUE
        TRAIN_XX[i, :, :, 3] = data_ndvi #NDVI
        TRAIN_XX[i, :, :, 4] = 1 - data[:, :, 12] / mid_slope #SLOPE
        TRAIN_XX[i, :, :, 5] = 1 - data[:, :, 13] / mid_elevation #ELEVATION


    with h5py.File(mask) as hdf:
        ls = list(hdf.keys())
        data=np.array(hdf.get('mask'))
        #plt.imshow(data)
        #plt.show()
        TRAIN_YY[i, :, :, 0] = data


In [ ]:
TRAIN_XX[np.isnan(TRAIN_XX)] = 0.000001
print(TRAIN_XX.min(), TRAIN_XX.max(), TRAIN_YY.min(), TRAIN_YY.max())

In [ ]:
def dice_loss(y_true, y_pred):
  y_true = tf.cast(y_true, tf.float32)
  y_pred = tf.math.sigmoid(y_pred)
  numerator = 2 * tf.reduce_sum(y_true * y_pred)
  denominator = tf.reduce_sum(y_true + y_pred)

  return 1 - (numerator / denominator)

In [ ]:
img=234
fig,(ax1,ax2, ax3, ax4, ax5)= plt.subplots(1,5,figsize=(15,10))


ax1.set_title("RGB image")
ax2.set_title("NDVI")
ax3.set_title("Slope")
ax4.set_title("Elevation")
ax5.set_title("Mask")
ax1.imshow(TRAIN_XX[img, :, :, 0:3])
ax2.imshow(TRAIN_XX[img, :, :, 3])
ax3.imshow(TRAIN_XX[img, :, :, 4])
ax4.imshow(TRAIN_XX[img, :, :, 5])
ax5.imshow(TRAIN_YY[img, :, :, 0])

Note: Learn about NDVI, Slope and Elevation properly, Something is going wrong in thw split!!

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data
x_train, x_valid, y_train, y_valid = train_test_split(TRAIN_XX, TRAIN_YY, test_size=0.2, shuffle= True, random_state=42)

In [ ]:
img=1548
fig,(ax1,ax2, ax3, ax4, ax5)= plt.subplots(1,5,figsize=(15,10))

ax1.set_title("RGB image")
ax2.set_title("NDVI")
ax3.set_title("Slope")
ax4.set_title("Elevation")
ax5.set_title("Mask")
ax1.imshow(x_train[img, :, :, 0:3])
ax2.imshow(x_train[img, :, :, 3])
ax3.imshow(x_train[img, :, :, 4])
ax4.imshow(x_train[img, :, :, 5])
ax5.imshow(y_train[img, :, :, 0])

In [ ]:
x_train.shape, y_train.shape

In [ ]:
# to release some memory, delete the unnecessary variable
del TRAIN_XX
del TRAIN_YY
del all_train
del all_mask

In [ ]:
img=1548
fig,(ax1,ax2, ax3, ax4)= plt.subplots(1,4,figsize=(15,10))


ax1.set_title("RGB image")
ax2.set_title("NDVI")
ax3.set_title("SLOPE")
ax4.set_title("Mask")
ax1.imshow(x_train[img, :, :, 0:3])
ax2.imshow(x_train[img, :, :, 3])
ax3.imshow(x_train[img, :, :, 4])
ax4.imshow(y_train[img, :, :, 0])

In [ ]:
from tensorflow.keras import backend as K

# recall
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

# precision
def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

#f1 score
def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation, Dropout
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# import numpy as np

# def encoder(input_shape):
#     inputs = Input(shape=input_shape)

#     # Encoder
#     conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
#     conv1 = BatchNormalization()(conv1)
#     conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
#     conv2 = BatchNormalization()(conv2)
#     conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
#     conv3 = BatchNormalization()(conv3)
#     conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     return Model(inputs, [pool1,pool2,pool3])

# def decoder(input_shape,encoded_features):
#     pool1,pool2,pool3 = encoded_features

#     up1 = UpSampling2D(size=(2, 2))(pool3)
#     merge1 = Concatenate()([pool2, up1])
#     conv4 = Conv2D(256, 3, activation='relu', padding='same')(merge1)

#     up2 = UpSampling2D(size=(2, 2))(conv4)
#     merge2 = Concatenate()([pool1, up2])
#     conv5 = Conv2D(128, 3, activation='relu', padding='same')(merge2)

#     up3 = UpSampling2D(size=(2, 2))(conv5)
#     conv6 = Conv2D(64, 3, activation='relu', padding='same')(up3)

#     decoded = Conv2D(1, (1,1), activation='sigmoid', padding='same')(conv6)
#     return decoded

# def landslide_detection_autoencoder(input_shape):
#     input_img = Input(shape=input_shape, name='input')

#     encoder_model = encoder(input_shape)
#     encoded = encoder_model(input_img)

#     decoded = decoder(input_shape, encoded)

#     return Model(inputs=input_img, outputs=decoded)

# # Training code for landslide detection using post images only
# input_shape = (128, 128, 6)  # Update this to match your post image dimensions
# model = landslide_detection_autoencoder(input_shape)

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

# model.summary()

# # history = model.fit(post_images,  # Input: post images only
# #                     landslide_labels,  # Target: landslide labels (binary)
# #                     batch_size=32,
# #                     epochs=135,
# #                     validation_split=0.2)


In [ ]:
# import tensorflow as tf
# from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation, Dropout
# from tensorflow.keras.models import Model
# from tensorflow.keras.optimizers import Adam
# import numpy as np

# def encoder(input_shape):
#     inputs = Input(shape=input_shape)

#     # Encoder
#     conv1 = Conv2D(64, 3, activation='relu', padding='same')(inputs)
#     conv1 = BatchNormalization()(conv1)
#     conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

#     conv2 = Conv2D(128, 3, activation='relu', padding='same')(pool1)
#     conv2 = BatchNormalization()(conv2)
#     conv2 = Conv2D(128, 3, activation='relu', padding='same')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

#     conv3 = Conv2D(256, 3, activation='relu', padding='same')(pool2)
#     conv3 = BatchNormalization()(conv3)
#     conv3 = Conv2D(256, 3, activation='relu', padding='same')(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

#     return Model(inputs, [pool1,pool2,pool3])

# def decoder(input_shape,encoded_features):
#     pool1,pool2,pool3 = encoded_features

#     up1 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(pool3)
#     merge1 = Concatenate()([pool2, up1])
#     conv4 = Conv2D(256, 3, activation='relu', padding='same')(merge1)

#     up2 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(conv4)
#     merge2 = Concatenate()([pool1, up2])
#     conv5 = Conv2D(128, 3, activation='relu', padding='same')(merge2)


#     up3 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv5)
#     conv6 = Conv2D(64, 3, activation='relu', padding='same')(up3)

#     decoded = Conv2D(1, (1,1), activation='sigmoid', padding='same')(conv6)
#     return decoded

# def landslide_detection_autoencoder(input_shape):
#     input_img = Input(shape=input_shape, name='input')

#     encoder_model = encoder(input_shape)
#     encoded = encoder_model(input_img)

#     decoded = decoder(input_shape, encoded)

#     return Model(inputs=input_img, outputs=decoded)

# # Training code for landslide detection using post images only
# input_shape = (128, 128, 6)  # Update this to match your post image dimensions
# model = landslide_detection_autoencoder(input_shape)

# opt = tf.keras.optimizers.Adam(learning_rate=0.001)

# model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

# model.summary()

In [ ]:
#With extra layers "FINAL".
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, BatchNormalization, Activation, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np

initializer = tf.keras.initializers.HeNormal()

def encoder(input_shape):
    inputs = Input(shape=input_shape)

    inputs = Input(shape=input_shape)

    conv0 = Conv2D(32, 3, activation='relu',  padding='same')(inputs)
    conv0 = Conv2D(32, 3, activation='relu',  padding='same')(conv0)
    conv0 = BatchNormalization()(conv0)
    conv0 = Dropout(0.2)(conv0)
    pool0 = MaxPooling2D(pool_size=(2, 2))(conv0)

    conv1 = Conv2D(64, 3, activation='relu',  padding='same')(pool0)
    conv1 = Conv2D(64, 3, activation='relu', padding='same')(conv1)
    conv1 = BatchNormalization()(conv1)
    conv1 = Dropout(0.2)(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation='relu',  padding='same')(pool1)
    conv2 = Conv2D(128, 3, activation='relu',  padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.2)(conv2)
    conv2 = Conv2D(128, 3, activation='relu',   padding='same')(conv2)
    conv2 = Conv2D(128, 3, activation='relu',  padding='same')(conv2)
    conv2 = BatchNormalization()(conv2)
    conv2 = Dropout(0.2)(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation='relu',  padding='same')(pool2)
    conv3 = Conv2D(256, 3, activation='relu',  padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.3)(conv3)
    conv3 = Conv2D(256, 3, activation='relu',  padding='same')(conv3)
    conv3 = Conv2D(256, 3, activation='relu',  padding='same')(conv3)
    conv3 = BatchNormalization()(conv3)
    conv3 = Dropout(0.3)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    # Add more convolutional layers as needed
    conv4 = Conv2D(512, 3, activation='relu',  padding='same')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(512, 3, activation='relu',  padding='same')(conv4)
    conv4 = BatchNormalization()(conv4)
    conv4 = Dropout(0.3)(conv4)


    return Model(inputs, [conv0,conv1,conv2,conv3,conv4])

def decoder(input_shape,encoded_features):
  conv0,conv1,conv2,conv3,conv4 = encoded_features

  up1 = tf.keras.layers.Conv2DTranspose(256, (3, 3), strides=(2, 2), padding='same')(conv4)
  merge2 = Concatenate()([conv3, up1])
  conv5 = Conv2D(256, 3, activation='relu',  padding='same')(merge2)
  conv5= Dropout(0.3)(conv5)
  conv5 = Conv2D(256, 3, activation='relu', padding='same')(conv5)


  up2 = tf.keras.layers.Conv2DTranspose(128, (3, 3), strides=(2, 2), padding='same')(conv5)
  merge3 = Concatenate()([conv2, up2])
  conv6 = Conv2D(128, 3, activation='relu',   padding='same')(merge3)
  conv6= Dropout(0.2)(conv6)
  conv6 = Conv2D(256, 3, activation='relu',  padding='same')(conv6)

  up3 = tf.keras.layers.Conv2DTranspose(64, (3, 3), strides=(2, 2), padding='same')(conv6)
  merge4 = Concatenate()([conv1, up3])
  conv7 = Conv2D(64, 3, activation='relu', padding='same')(merge4)
  conv7= Dropout(0.1)(conv7)
  conv7 = Conv2D(64, 3, activation='relu',  padding='same')(conv7)

  up4 = tf.keras.layers.Conv2DTranspose(32, (3, 3), strides=(2, 2), padding='same')(conv7)
  merge5 = Concatenate()([conv0, up4])
  conv8 = Conv2D(32, 3, activation='relu',  padding='same')(merge5)
  conv8= Dropout(0.1)(conv8)
  conv8 = Conv2D(32, 3, activation='relu',  padding='same')(conv8)

  decoded = Conv2D(1, (1,1), activation='sigmoid', padding='same')(conv8)
  return decoded

def landslide_detection_autoencoder(input_shape):
    input_img = Input(shape=input_shape, name='input')

    encoder_model = encoder(input_shape)
    encoded = encoder_model(input_img)

    decoded = decoder(input_shape, encoded)

    return Model(inputs=input_img, outputs=decoded)

# Training code for landslide detection using post images only
input_shape = (128, 128, 6)  # Update this to match your post image dimensions
model = landslide_detection_autoencoder(input_shape)

opt = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

model.summary()

In [ ]:
# def unet_model(IMG_WIDTH, IMG_HIGHT, IMG_CHANNELS):
#     inputs = tf.keras.layers.Input((IMG_WIDTH, IMG_HIGHT, IMG_CHANNELS))

#     # Converted inputs to floating
#     #s = tf.keras.layers.Lambda(lambda x: x / 255)(inputs)


#     #Contraction path
#     c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(inputs)
#     c1 = tf.keras.layers.Dropout(0.1)(c1)
#     c1 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c1)
#     p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

#     c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p1)
#     c2 = tf.keras.layers.Dropout(0.1)(c2)
#     c2 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c2)
#     p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

#     c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p2)
#     c3 = tf.keras.layers.Dropout(0.2)(c3)
#     c3 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c3)
#     p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

#     c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p3)
#     c4 = tf.keras.layers.Dropout(0.2)(c4)
#     c4 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c4)
#     p4 = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(c4)

#     c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(p4)
#     c5 = tf.keras.layers.Dropout(0.3)(c5)
#     c5 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c5)

#     #Expansive path
#     u6 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c5)
#     u6 = tf.keras.layers.concatenate([u6, c4])
#     c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u6)
#     c6 = tf.keras.layers.Dropout(0.2)(c6)
#     c6 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c6)

#     u7 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c6)
#     u7 = tf.keras.layers.concatenate([u7, c3])
#     c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u7)
#     c7 = tf.keras.layers.Dropout(0.2)(c7)
#     c7 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c7)

#     u8 = tf.keras.layers.Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same')(c7)
#     u8 = tf.keras.layers.concatenate([u8, c2])
#     c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u8)
#     c8 = tf.keras.layers.Dropout(0.1)(c8)
#     c8 = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c8)

#     u9 = tf.keras.layers.Conv2DTranspose(16, (2, 2), strides=(2, 2), padding='same')(c8)
#     u9 = tf.keras.layers.concatenate([u9, c1], axis=3)
#     c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(u9)
#     c9 = tf.keras.layers.Dropout(0.1)(c9)
#     c9 = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', kernel_initializer='he_normal', padding='same')(c9)

#     outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)

#     model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', f1_m, precision_m, recall_m])

#     return model

In [ ]:
#model = unet_model(128, 128, 6)
# model.summary()
checkpointer = tf.keras.callbacks.ModelCheckpoint("best_model.h5", monitor="val_f1_m", verbose=1, save_best_only=True, mode="max")
# earlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_f1_m', patience=10, verbose=1, mode='max')

callbacks = [
    # earlyStopping,
    checkpointer
    ]
history = model.fit(x_train, y_train, batch_size=16,
          epochs=130,
          verbose = 2,
          validation_data=(x_valid, y_valid),
          callbacks=callbacks)

model.save("model_save.h5")

In [ ]:
loss, accuracy, f1_score, precision, recall = model.evaluate(x_valid, y_valid, verbose=0)
print(loss, accuracy, f1_score, precision, recall)

In [ ]:
fig,((ax11,ax12),(ax13,ax14)) = plt.subplots(2,2,figsize=(20,15))
ax11.plot(history.history['loss'])
ax11.plot(history.history['val_loss'])
ax11.title.set_text('model loss')
ax11.set_ylabel('loss')
ax11.set_xlabel('epoch')
ax11.legend(['train', 'validation'], loc='upper left')

ax12.plot(history.history['precision_m'])
ax12.plot(history.history['val_precision_m'])
ax12.set_title('model precision')
ax12.set_ylabel('precision')
ax12.set_xlabel('epoch')
ax12.legend(['train', 'validation'], loc='upper left')

ax13.plot(history.history['recall_m'])
ax13.plot(history.history['val_recall_m'])
ax13.set_title('model recall')
ax13.set_ylabel('recall')
ax13.set_xlabel('epoch')
ax13.legend(['train', 'validation'], loc='upper left')

ax14.plot(history.history['f1_m'])
ax14.plot(history.history['val_f1_m'])
ax14.set_title('model f1')
ax14.set_ylabel('f1')
ax14.set_xlabel('epoch')
ax14.legend(['train', 'validation'], loc='upper left')

In [ ]:
# threshold = 0.2
# pred_img = model.predict(x_valid)
# #print(pred_img)
# pred_img = (pred_img > threshold).astype(np.uint8)
# #print(pred_img[0])

In [ ]:
# img = 115
# fig,(ax1,ax2,ax3)= plt.subplots(1,3,figsize=(15,10))
# ax1.imshow(pred_img[img, :, :, 0])
# ax1.set_title("Predictions")
# ax2.imshow(y_valid[img, :, :, 0])
# ax2.set_title("Label")
# ax3.imshow(x_valid[img, :, :, 0:3])
# ax3.set_title('Training Image')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'pred_img' contains prediction maps, 'y_valid' contains labels, and 'x_valid' contains training images.

img = 119

# Assuming you have already thresholded 'pred_img' to obtain a binary mask.
# If not, you can threshold it as mentioned in the previous answer.

prediction_binary_mask = pred_img[img, :, :, 0]

# Create a colormap for highlighting
cmap = plt.cm.get_cmap("viridis")  # You can choose a different colormap

# Create subplots
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 10))

# Display predictions
ax1.imshow(pred_img[img, :, :, 0], cmap=cmap)
ax1.set_title("Predictions")

# Display labels
ax2.imshow(y_valid[img, :, :, 0], cmap=cmap)
ax2.set_title("Label")

# Display the training image with highlighted regions
training_image = x_valid[img, :, :, 0:3].copy()
training_image[prediction_binary_mask == 1] = [255, 0, 0]  # Highlight in red
ax3.imshow(training_image)
ax3.set_title("Training Image with Highlighted Regions")

# Show the plots
plt.show()
